In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count,countDistinct,split,sum as spark_sum
from pyspark.sql.window import Window
from pyspark.sql.functions import max as spark_max, col, rank
spark = SparkSession.builder.appName("Google NGram Example").getOrCreate()

In [ ]:
columns = ["ngram", "year", "count", "pages", "books"]

df = spark.read.csv("sample_data/ngram.csv", sep="\t", header=False, inferSchema=True).toDF(*columns)
df.show(5)
df.printSchema()


+--------+----+-----+-----+-----+
|   ngram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
+--------+----+-----+-----+-----+
only showing top 5 rows

root
 |-- ngram: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- pages: integer (nullable = true)
 |-- books: integer (nullable = true)



In [ ]:
df.createOrReplaceTempView("ngrams")


## **1) Retourner tous les bi grammes dont le nombre Count est supérieur à cinq.**

In [ ]:
spark.sql("SELECT * FROM ngrams WHERE count > 5").show()


+--------+----+-----+-----+-----+
|   ngram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



In [ ]:
df.filter(df["count"] > 5).show()

+--------+----+-----+-----+-----+
|   ngram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+



In [ ]:
spark.sql("SELECT year, COUNT(*) AS total_bigrams FROM ngrams GROUP BY year ORDER BY year").show()

+----+-------------+
|year|total_bigrams|
+----+-------------+
|1780|            1|
|1803|            1|
|1806|            1|
|1823|            1|
|1824|            1|
|1825|            1|
|1829|            1|
|1830|            1|
|1831|            1|
|1833|            1|
|1834|            1|
|1836|            1|
|1839|            1|
|1840|            1|
|1841|            1|
|1845|            1|
|1847|            1|
|1848|            1|
|1856|            1|
|1857|            1|
+----+-------------+
only showing top 20 rows



In [ ]:
df.groupBy("Year").count().withColumnRenamed("count", "total_bigrams").orderBy("Year").show()

+----+-------------+
|Year|total_bigrams|
+----+-------------+
|1780|            1|
|1803|            1|
|1806|            1|
|1823|            1|
|1824|            1|
|1825|            1|
|1829|            1|
|1830|            1|
|1831|            1|
|1833|            1|
|1834|            1|
|1836|            1|
|1839|            1|
|1840|            1|
|1841|            1|
|1845|            1|
|1847|            1|
|1848|            1|
|1856|            1|
|1857|            1|
+----+-------------+
only showing top 20 rows



## **3) Retourner les bi grammes qui ont le plus grand nombre de count dans chaque année.**

In [ ]:
spark.sql("""
SELECT year, ngram, count
FROM ngrams n1
WHERE count = (
    SELECT MAX(count) FROM ngrams n2 WHERE n1.year = n2.year
) ORDER BY year
""").show()

+----+-----+-----+
|year|ngram|count|
+----+-----+-----+
|1780| ! 09|    1|
|1803| ! 09|    1|
|1806| ! 09|    1|
|1823| ! 09|    1|
|1824| ! 09|    1|
|1825| ! 09|    1|
|1829| ! 09|    3|
|1830| ! 09|    2|
|1831| ! 09|    1|
|1833| ! 09|    1|
|1834| ! 09|    4|
|1836| ! 09|    1|
|1839| ! 09|    1|
|1840| ! 09|    1|
|1841| ! 09|    2|
|1845| ! 09|    1|
|1847| ! 09|    2|
|1848| ! 09|    1|
|1856| ! 09|    1|
|1857| ! 09|    2|
+----+-----+-----+
only showing top 20 rows



In [ ]:
window = Window.partitionBy("year").orderBy(col("count").desc())
df.withColumn("rank", rank().over(window)) \
  .filter(col("rank") == 1) \
  .select("year", "ngram", "count") \
  .show()

+----+-----+-----+
|year|ngram|count|
+----+-----+-----+
|1780| ! 09|    1|
|1803| ! 09|    1|
|1806| ! 09|    1|
|1823| ! 09|    1|
|1824| ! 09|    1|
|1825| ! 09|    1|
|1829| ! 09|    3|
|1830| ! 09|    2|
|1831| ! 09|    1|
|1833| ! 09|    1|
|1834| ! 09|    4|
|1836| ! 09|    1|
|1839| ! 09|    1|
|1840| ! 09|    1|
|1841| ! 09|    2|
|1845| ! 09|    1|
|1847| ! 09|    2|
|1848| ! 09|    1|
|1856| ! 09|    1|
|1857| ! 09|    2|
+----+-----+-----+
only showing top 20 rows



## **4) Retourner tous les bi grammes qui sont apparus dans 20 années différentes.**

In [ ]:
spark.sql("""
SELECT ngram, COUNT(DISTINCT year) AS nb_years
FROM ngrams
GROUP BY ngram
HAVING nb_years = 20
""").show()

+--------+--------+
|   ngram|nb_years|
+--------+--------+
|! $17.95|      20|
+--------+--------+



In [ ]:
df.groupBy("ngram").agg(countDistinct("year").alias("nb_years")) \
  .filter(col("nb_years") == 20).show()

+--------+--------+
|   ngram|nb_years|
+--------+--------+
|! $17.95|      20|
+--------+--------+



## **5) Retourner tous les bi grammes qui contiennent ! le caractère dans la première partie et le caractère 9 dans la deuxième partie (les deux parties sont séparées par un espace).**

In [ ]:
spark.sql("""
SELECT *
FROM ngrams
WHERE split(ngram, ' ')[0] LIKE '%!%'
  AND split(ngram, ' ')[1] LIKE '%9%'
""").show()


+--------+----+-----+-----+-----+
|   ngram|year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
|! $17.95|1993|    2|    2|    2|
|! $17.95|1995|    1|    1|    1|
|! $17.95|1996|    4|    2|    2|
|! $17.95|1997|    6|    5|    5|
|! $17.95|1998|    4|    3|    3|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2001|    5|    4|    4|
|! $17.95|2002|    5|    5|    5|
|! $17.95|2003|    2|    2|    2|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|! $17.95|2006|    5|    5|    5|
|! $17.95|2007|    2|    2|    2|
|! $17.95|2008|    2|    2|    2|
+--------+----+-----+-----+-----+
only showing top 20 rows



In [ ]:
parts = split(col("ngram"), " ")
df.filter(
    (parts.getItem(0).contains("!")) &
    (parts.getItem(1).contains("9"))
).show(truncate=False)

+--------+----+-----+-----+-----+
|ngram   |year|count|pages|books|
+--------+----+-----+-----+-----+
|! $17.95|1985|1    |1    |1    |
|! $17.95|1987|1    |1    |1    |
|! $17.95|1990|1    |1    |1    |
|! $17.95|1991|1    |1    |1    |
|! $17.95|1992|5    |5    |5    |
|! $17.95|1993|2    |2    |2    |
|! $17.95|1995|1    |1    |1    |
|! $17.95|1996|4    |2    |2    |
|! $17.95|1997|6    |5    |5    |
|! $17.95|1998|4    |3    |3    |
|! $17.95|1999|11   |10   |10   |
|! $17.95|2000|11   |9    |9    |
|! $17.95|2001|5    |4    |4    |
|! $17.95|2002|5    |5    |5    |
|! $17.95|2003|2    |2    |2    |
|! $17.95|2004|14   |14   |14   |
|! $17.95|2005|13   |13   |13   |
|! $17.95|2006|5    |5    |5    |
|! $17.95|2007|2    |2    |2    |
|! $17.95|2008|2    |2    |2    |
+--------+----+-----+-----+-----+
only showing top 20 rows



## **6) Retourner les bi grammes qui sont apparus dans toutes les années présentes dans les données.**

In [ ]:
spark.sql("""
SELECT ngram, COUNT(DISTINCT year) AS nb_years
FROM ngrams
GROUP BY ngram
HAVING COUNT(DISTINCT year) = (SELECT COUNT(DISTINCT year) FROM ngrams)
""").show()

+-----+--------+
|ngram|nb_years|
+-----+--------+
| ! 09|     100|
+-----+--------+



In [ ]:
total_years = df.selectExpr("COUNT(DISTINCT year) as nb").collect()[0]["nb"]

df.groupBy("ngram").agg(countDistinct("year").alias("nb_years")) \
  .filter(col("nb_years") == total_years).show()

+-----+--------+
|ngram|nb_years|
+-----+--------+
| ! 09|     100|
+-----+--------+



## **7) Retourner le nombre total de pages et de livres dans lesquels chaque bi-gramme apparaît pour chaque année disponible, trié par ordre alphabétique.**

In [ ]:
spark.sql("""
SELECT ngram, year, SUM(pages) AS total_pages, SUM(books) AS total_books
FROM ngrams
GROUP BY ngram, year
ORDER BY ngram ASC
""").show()

+--------+----+-----------+-----------+
|   ngram|year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



In [ ]:
df.groupBy("ngram", "year") \
  .agg(spark_sum("pages").alias("total_pages"),
       spark_sum("books").alias("total_books")) \
  .orderBy("ngram") \
  .show()

+--------+----+-----------+-----------+
|   ngram|year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



## **8) Retourner le nombre total de bi grammes différents dans chaque année, triés par ordre décroissant de l'année.**

In [ ]:
spark.sql("SELECT year DISTINCT,COUNT(ngram) AS total_ngram FROM ngrams GROUP BY year ORDER BY year DESC").show()

+--------+-----------+
|DISTINCT|total_ngram|
+--------+-----------+
|    2008|          2|
|    2007|          2|
|    2006|          2|
|    2005|          2|
|    2004|          2|
|    2003|          2|
|    2002|          2|
|    2001|          2|
|    2000|          2|
|    1999|          2|
|    1998|          2|
|    1997|          2|
|    1996|          2|
|    1995|          2|
|    1993|          2|
|    1992|          2|
|    1991|          2|
|    1990|          2|
|    1987|          2|
|    1985|          2|
+--------+-----------+
only showing top 20 rows



In [ ]:
df.groupBy("Year") \
  .agg(countDistinct("ngram").alias("distinct_bigrams")) \
  .orderBy(col("Year").desc()) \
  .show()

+----+----------------+
|Year|distinct_bigrams|
+----+----------------+
|2008|               2|
|2007|               2|
|2006|               2|
|2005|               2|
|2004|               2|
|2003|               2|
|2002|               2|
|2001|               2|
|2000|               2|
|1999|               2|
|1998|               2|
|1997|               2|
|1996|               2|
|1995|               2|
|1993|               2|
|1992|               2|
|1991|               2|
|1990|               2|
|1987|               2|
|1985|               2|
+----+----------------+
only showing top 20 rows

